In [2]:
"""
IMPORTS
"""

import pandas as pd
import numpy as np
import onnxruntime as rt

from interpret.ext.blackbox import TabularExplainer
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.blackbox import PFIExplainer 
from interpret.ext.glassbox import DecisionTreeExplainableModel
from raiwidgets import ExplanationDashboard
import dice_ml
from dice_ml.utils import helpers 

from utils.dataloader import DataLoader
from models.modelloader import ModelLoader
from utils.constants import protected_attributes, group_proxies


from tests.metrics import MetricsTester
from tests.mututation import MutationTester
from tests.metamorphic import MetamorphicTester
from tests.shapley import ShapleyTester

Initializing test: data_shuffler...
Initializing test: data_remover...
Initializing test: data_repetition...
Initializing test: label_error...
Initializing test: feature_remover...


In [3]:
"""
    INTERESTING LINKS:
        - https://dl.acm.org/doi/pdf/10.1145/3338906.3338937
        - INTERPRET LIBRARY https://github.com/interpretml/interpret-community
"""

'\n    INTERESTING LINKS:\n        - https://dl.acm.org/doi/pdf/10.1145/3338906.3338937\n        - INTERPRET LIBRARY https://github.com/interpretml/interpret-community\n'

In [4]:
"""
SET MODEL(S)
"""
MODEL_UNDER_TEST = '.\..\models\\audit_models\\model_2.onnx'

In [5]:
"""
LOAD DATASET
"""

dataloader = DataLoader()
X_train, y_train = dataloader.load_split('train')
print(f'{X_train.shape}, {y_train.shape}')
X_test, y_test = dataloader.load_split('test')
print(f'{X_test.shape}, {y_test.shape}')
X_data, y_data = dataloader.load_split('full')
print(f'{X_data.shape}, {y_data.shape}')
dataset = pd.concat([X_data, y_data], axis=1)
print(f'{dataset.shape}')

(10116, 315), (10116,)
(2529, 315), (2529,)
(12645, 315), (12645,)
(12645, 316)


In [6]:

params = {
    'onnx_model_path': MODEL_UNDER_TEST
}

model = ModelLoader.load_model(type='onnx', params=params)

y_pred = model.predict(X_data)
y_pred_proba = model.predict_proba(X_test=X_data)

In [7]:
"""
FUNCTIONAL TESTING
"""
metrics = MetricsTester(protected_variables=group_proxies)


In [8]:
metrics.get_metrics_overall(y_true=y_data, y_pred=y_pred, sensitive_features=X_data[group_proxies[0]])

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default b

TN-FP-FN-TP    (6061, 5319, 29, 1236)
acc                          0.577066
prec                         0.188558
rec                          0.977075
f1                           0.316113
fnr                          0.022925
fpr                          0.467399
sel                          0.518387
count                           12645
dtype: object

In [12]:
"""
TODO: Fix bug Nonetype for index in (1, 3, 4) 
"""
index = 0

metrics.get_metrics_by_group(y_true=y_data, y_pred=y_pred, sensitive_features=X_data[group_proxies[4]])

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default b

,fnr,fpr,sel,count
relatie_partner_aantal_partner___partner__gehuwd_,,,,
0,0.023445,0.461043,0.512453,9837
1,0.021201,0.489885,0.53923,2804
2,0.0,0.333333,0.5,4


In [ ]:
"""
METAMORPHIC TESTING: See model 1 results in results/results_model_1_metamorphic_testing_column_avg.txt
"""
run_metamorphic = True

if run_metamorphic:
    metamorphic_tester = MetamorphicTester(X=dataset, model_path=MODEL_UNDER_TEST)
    metamorphic_tester.test()
    metamorphic_tester.analyse_test()


In [ ]:
if run_metamorphic:
    print('global average: ', metamorphic_tester.global_average)
    print(metamorphic_tester.column_avg)

In [ ]:
metamorphic_tester.column_avg

In [ ]:
"""
MUTATION TESTING
"""

params = {
    'data_shuffler': 1000,
    'data_remover': 0.25,
    'data_repetition': 1000, 
    'label_error': 1000,
    'feature_remover': 31,
}


mutator = MutationTester(y_pred_baseline=y_pred, df_train=X_train, df_test=X_test)
mutation_score = mutator.test_mutants(params)
print(f"Mutation score: {mutation_score}")

In [ ]:
"""
SHAP: See model 1 results in results/results_model_1_ranked_values_classes.txt and results_model_1_global_explanation_feature_importance.txt
"""

run_shapley = True
if run_shapley:

    # 1. Using SHAP TabularExplainer
    explainer = TabularExplainer(model, 
                                X_data, 
                                features=X_data.columns, 
                                classes=['fraud', 'no fraud'])

    # Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
    # x_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
    global_explanation = explainer.explain_global(X_data)


    # Sorted SHAP values
    print('ranked global importance values: {}'.format(global_explanation.get_ranked_global_values()))
    # Corresponding feature names
    print('ranked global importance names: {}'.format(global_explanation.get_ranked_global_names()))
    # Feature ranks (based on original order of features)
    print('global importance rank: {}'.format(global_explanation.global_importance_rank))

    # Note: Do not run this cell if using PFIExplainer, it does not support per class explanations
    # Per class feature names
    print('ranked per class feature names: {}'.format(global_explanation.get_ranked_per_class_names()))
    # Per class feature importance values
    print('ranked per class feature values: {}'.format(global_explanation.get_ranked_per_class_values()))

In [ ]:
# Print out a dictionary that holds the sorted feature importance names and values
print('global importance rank: {}'.format(global_explanation.get_feature_importance_dict()))

In [ ]:
# feature shap values for all features and all data points in the training data
print('local importance values: {}'.format(global_explanation.local_importance_values))

In [ ]:
# Note: Do not run this cell if using PFIExplainer, it does not support local explanations
# You can pass a specific data point or a group of data points to the explain_local function

# E.g., Explain the first data point in the test set
instance_num = 0

instance = dataset.iloc[instance_num, :]
instance = dataset.iloc[instance_num,:]
instance_df = pd.DataFrame(instance, X_test.columns).T


local_explanation = explainer.explain_local(instance_df)

In [ ]:
# Get the prediction for the first member of the test set and explain why model made that prediction
prediction_value = model.predict(X_test)[instance_num]

sorted_local_importance_values = local_explanation.get_ranked_local_values()[prediction_value]
sorted_local_importance_names = local_explanation.get_ranked_local_names()[prediction_value]

print('local importance values: {}'.format(sorted_local_importance_values))
print('local importance names: {}'.format(sorted_local_importance_names))

In [ ]:
ExplanationDashboard(global_explanation, model, dataset=X_data, true_y=y_data)


In [16]:
"""
COUNTERFACTUALS
"""

# Model

data = dice_ml.Data(dataframe=dataset, 
                   continuous_features=["persoon_leeftijd_bij_onderzoek", "adres_dagen_op_adres"],
                   outcome_name="checked")

rf_dice = dice_ml.Model(model=model, 
                        # There exist backends for tf, torch, ...
                        backend="sklearn")

explainer = dice_ml.Dice(data, 
                         rf_dice, 
                         # Random sampling, genetic algorithm, kd-tree,...
                         method="random")


In [18]:
original_input = X_test[11:12]
cf_examples = explainer.generate_counterfactuals(original_input,
                                           total_CFs=5,
                                           desired_class="opposite")
cf_examples.visualize_as_dataframe(show_only_changes=True)

  0%|          | 0/1 [00:00<?, ?it/s]

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '570' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with int64, please explicitly cast to a co

Query instance (original outcome : 0)


,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,adres_recentste_buurt_vreewijk,adres_recentste_plaats_other,adres_recentste_plaats_rotterdam,adres_recentste_wijk_charlois,adres_recentste_wijk_delfshaven,adres_recentste_wijk_feijenoord,adres_recentste_wijk_ijsselmonde,adres_recentste_wijk_kralingen_c,adres_recentste_wijk_noord,adres_recentste_wijk_other,adres_recentste_wijk_prins_alexa,adres_recentste_wijk_stadscentru,adres_unieke_wijk_ratio,afspraak_aanmelding_afgesloten,afspraak_aantal_woorden,afspraak_afgelopen_jaar_afsprakenplan,afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel,afspraak_afgelopen_jaar_ontheffing,afspraak_afgelopen_jaar_ontheffing_taaleis,afspraak_afgelopen_jaar_plan_van_aanpak,afspraak_afgelopen_jaar_signaal_voor_medewerker,afspraak_afgelopen_jaar_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_afgelopen_jaar_voortgang_aanmelding_en_deelname,afspraak_afsprakenplan,afspraak_controle_aankondiging_maatregel,afspraak_controle_verwijzing,afspraak_deelname_compleet_uit_webapplicatie,afspraak_galo_gesprek,afspraak_gespr__einde_zoekt___galo_gesprek_,afspraak_inspanningsperiode,afspraak_laatstejaar_aantal_woorden,afspraak_laatstejaar_resultaat_ingevuld,afspraak_laatstejaar_resultaat_ingevuld_uniek,afspraak_other,afspraak_participatietrede_vervolgmeting,afspraak_resultaat_ingevuld_uniek,afspraak_signaal_van_aanbieder,afspraak_signaal_voor_medewerker,afspraak_toevoegen_inschrijving_uwvwb,afspraak_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_verzenden_beschikking_i_v_m__niet_voldoen_aan_wet_taaleis,afspraak_voortgangsgesprek,belemmering_aantal_huidig,belemmering_dagen_financiele_problemen,belemmering_dagen_lichamelijke_problematiek,belemmering_dagen_psychische_problemen,belemmering_financiele_problemen,belemmering_hist_lichamelijke_problematiek,belemmering_hist_psychische_problemen,belemmering_hist_stabiele_mix__sz____dagbesteding_werk,belemmering_hist_taal,belemmering_hist_verslavingsproblematiek,belemmering_ind,belemmering_ind_hist,belemmering_niet_computervaardig,belemmering_psychische_problemen,belemmering_woonsituatie,beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie,beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_huidig_bekend,beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie,competentie_aansturen,competentie_analyseren,competentie_ethisch_en_integer_handelen,competentie_formuleren_en_rapporteren,competentie_gedrevenheid_en_ambitie_tonen,competentie_instructies_en_procedures_opvolgen,competentie_kwaliteit_leveren,competentie_leren,competentie_materialen_en_middelen_inzetten,competentie_met_druk_en_tegenslag_omgaan,competentie_omgaan_met_verandering_en_aanpassen,competentie_onderzoeken,competentie_op_de_behoeften_en_verwachtingen_van_de__klant__richten,competentie_other,competentie_overtuigen_en_beïnvloeden,competentie_plannen_en_organiseren,competentie_samenwerken_en_overleggen,competentie_vakdeskundigheid_toepassen,contacten_onderwerp__arbeids_motivatie,contacten_onderwerp__pre__intake,contacten_onderwerp__werk_intake,contacten_onderwerp_arbeidsdiagnose_dariuz,contacten_onderwerp_beoordelen_taaleis,contacten_onderwerp_boolean__arbeids_motivatie,contacten_onderwerp_boolean__pre__intake,contacten_onderwerp_boolean__werk_intake,contacten_onderwerp_boolean_beoordelen_taaleis,contacten_onderwerp_boolean_contact_derden,contacten_onderwerp_boolean_contact_met_aanbieder,contacten_onderwerp_boolean_diagnosegesprek,contacten_onderwerp_boolean_documenten__innemen_,contacten_onderwerp_


Diverse Counterfactual set (new outcome: 1)


,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,adres_recentste_buurt_vreewijk,adres_recentste_plaats_other,adres_recentste_plaats_rotterdam,adres_recentste_wijk_charlois,adres_recentste_wijk_delfshaven,adres_recentste_wijk_feijenoord,adres_recentste_wijk_ijsselmonde,adres_recentste_wijk_kralingen_c,adres_recentste_wijk_noord,adres_recentste_wijk_other,adres_recentste_wijk_prins_alexa,adres_recentste_wijk_stadscentru,adres_unieke_wijk_ratio,afspraak_aanmelding_afgesloten,afspraak_aantal_woorden,afspraak_afgelopen_jaar_afsprakenplan,afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel,afspraak_afgelopen_jaar_ontheffing,afspraak_afgelopen_jaar_ontheffing_taaleis,afspraak_afgelopen_jaar_plan_van_aanpak,afspraak_afgelopen_jaar_signaal_voor_medewerker,afspraak_afgelopen_jaar_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_afgelopen_jaar_voortgang_aanmelding_en_deelname,afspraak_afsprakenplan,afspraak_controle_aankondiging_maatregel,afspraak_controle_verwijzing,afspraak_deelname_compleet_uit_webapplicatie,afspraak_galo_gesprek,afspraak_gespr__einde_zoekt___galo_gesprek_,afspraak_inspanningsperiode,afspraak_laatstejaar_aantal_woorden,afspraak_laatstejaar_resultaat_ingevuld,afspraak_laatstejaar_resultaat_ingevuld_uniek,afspraak_other,afspraak_participatietrede_vervolgmeting,afspraak_resultaat_ingevuld_uniek,afspraak_signaal_van_aanbieder,afspraak_signaal_voor_medewerker,afspraak_toevoegen_inschrijving_uwvwb,afspraak_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_verzenden_beschikking_i_v_m__niet_voldoen_aan_wet_taaleis,afspraak_voortgangsgesprek,belemmering_aantal_huidig,belemmering_dagen_financiele_problemen,belemmering_dagen_lichamelijke_problematiek,belemmering_dagen_psychische_problemen,belemmering_financiele_problemen,belemmering_hist_lichamelijke_problematiek,belemmering_hist_psychische_problemen,belemmering_hist_stabiele_mix__sz____dagbesteding_werk,belemmering_hist_taal,belemmering_hist_verslavingsproblematiek,belemmering_ind,belemmering_ind_hist,belemmering_niet_computervaardig,belemmering_psychische_problemen,belemmering_woonsituatie,beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie,beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_huidig_bekend,beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie,competentie_aansturen,competentie_analyseren,competentie_ethisch_en_integer_handelen,competentie_formuleren_en_rapporteren,competentie_gedrevenheid_en_ambitie_tonen,competentie_instructies_en_procedures_opvolgen,competentie_kwaliteit_leveren,competentie_leren,competentie_materialen_en_middelen_inzetten,competentie_met_druk_en_tegenslag_omgaan,competentie_omgaan_met_verandering_en_aanpassen,competentie_onderzoeken,competentie_op_de_behoeften_en_verwachtingen_van_de__klant__richten,competentie_other,competentie_overtuigen_en_beïnvloeden,competentie_plannen_en_organiseren,competentie_samenwerken_en_overleggen,competentie_vakdeskundigheid_toepassen,contacten_onderwerp__arbeids_motivatie,contacten_onderwerp__pre__intake,contacten_onderwerp__werk_intake,contacten_onderwerp_arbeidsdiagnose_dariuz,contacten_onderwerp_beoordelen_taaleis,contacten_onderwerp_boolean__arbeids_motivatie,contacten_onderwerp_boolean__pre__intake,contacten_onderwerp_boolean__werk_intake,contacten_onderwerp_boolean_beoordelen_taaleis,contacten_onderwerp_boolean_contact_derden,contacten_onderwerp_boolean_contact_met_aanbieder,contacten_onderwerp_boolean_diagnosegesprek,contacten_onderwerp_boolean_documenten__innemen_,contacten_onderwerp_

In [ ]:
"""
FAT FORENSICS
"""

# # 2. Using MimicExplainer
# augment_data is optional and if true, oversamples the initialization examples to improve surrogate model accuracy to fit original model.  Useful for high-dimensional data where the number of rows is less than the number of columns. 
# max_num_of_augmentations is optional and defines max number of times we can increase the input data size.
# LGBMExplainableModel can be replaced with LinearExplainableModel, SGDExplainableModel, or DecisionTreeExplainableModel
# explainer = MimicExplainer(model, 
#                            X_data, 
#                            DecisionTreeExplainableModel, 
#                            augment_data=True, 
#                            max_num_of_augmentations=10, 
#                            features=X_data.columns, 
#                            classes=['checked'],
#                            labels=len(X_data))





# 3. Using PFIExplainer

# Use the parameter "metric" to pass a metric name or function to evaluate the permutation. 
# Note that if a metric function is provided a higher value must be better.
# Otherwise, take the negative of the function or set the parameter "is_error_metric" to True.
# Default metrics: 
# F1 Score for binary classification, F1 Score with micro average for multiclass classification and
# Mean absolute error for regression

# explainer = PFIExplainer(model, 
#                          features=X_data.columns, 
#                          classes=['checked'])